---
title: "Procesamiento de Señales e Imagenes"
description: "PSIM -- 101849"
subtitle: "Ingeniería Biomédica"
lang: es
author: "Ph.D. Pablo Eduardo Caicedo Rodríguez"
date: last-modified
format:
  revealjs: 
    code-tools: true
    code-overflow: wrap
    code-line-numbers: true
    code-copy: true
    fig-align: center
    self-contained: true
    theme: 
      - simple
      - ../../recursos/estilos/metropolis.scss
    slide-number: true
    preview-links: auto
    logo: ../../recursos/imagenes/generales/Escuela_Rosario_logo.png
    css: ../../recursos/estilos/styles_pres.scss
    footer: <https://pablocaicedor.github.io/>
    transition: fade
    progress: true
    scrollable: true
    mainfont: "Fira Code"
---

## 1) Título y propósito

**Procesamiento de señales e imágenes**
*Cómo transformar datos del mundo (sonidos, fotos, mediciones) en información útil para la vida diaria y la salud.*

**Preguntas guía**

* ¿Qué es una *señal* en la vida cotidiana?
* ¿Qué es una *imagen* y cómo la entiende un computador?
* ¿Por qué *procesarlas* mejora fotos, audios y ayuda en medicina?

:::{.notes}
Guion: Romper el hielo preguntando: “¿Qué señales usan a diario sin darse cuenta?” (Wi-Fi, música, pulso). Explicar que hoy “miraremos” y “escucharemos” datos como lo hace una computadora.
Dinámica: Levanten la mano si alguna vez una foto les salió borrosa y el teléfono la “arregló”. ¿Qué pasó detrás?
:::

---

## 2) ¿Qué es una señal?

Una **señal** es *información que cambia con el tiempo* o con otra variable.
Ejemplos intuitivos:

* **Audio/voz:** la amplitud del sonido sube y baja.
* **Temperatura ambiental:** varía a lo largo del día.
* **Pulso cardíaco:** latido a latido, cuenta una historia del cuerpo.

:::{.notes}
Analogía: La señal es como un cuento por capítulos; cada instante agrega una línea a la historia. Objetivo: que reconozcan señales a su alrededor (semáforos, brillo de la pantalla, ritmo al caminar).
:::

---

## 3) ¿Qué es una imagen?

Una **imagen digital** es una “rejilla” de puntitos llamados **píxeles**.
Cada píxel guarda *brillo* y *color*.
Al cambiar estos valores de forma ordenada, aparece una foto reconocible.

:::{.notes}
Analogía: Piensa en mosaicos: cada baldosa aporta un color; juntas forman una figura. Objetivo: entender que “ver” para la máquina es leer números en una tabla.
:::

---

## 4) ¿Por qué procesarlas?

**Cotidiano**

* Hacer fotos más **nítidas** y menos **ruidosas**.
* **Comprimir** para enviar más rápido sin que se note la diferencia.
* **Reconocer patrones**: voz, rostros, gestos simples.

**Salud**

* **ECG/EEG:** ver ritmos y detectar señales de alerta.
* **Imágenes médicas (rayos-X, TAC, RM, ecografía):** apoyar diagnóstico y guiar tratamientos.

:::{.notes}
Pregunta detonante: ¿Qué prefieren, un médico mirando una imagen “cruda” o una imagen mejorada que resalte detalles importantes? Introducir idea de apoyo, no reemplazo, a profesionales.
:::

---

## 5) Demostraciones rápidas (Python)

### 5.1 Voz sintética: señal seno con ruido + “espectro” simple

In [ ]:
# | fig-cap: "Señal tipo voz: tono de 440 Hz con ruido."
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

Fs = 8000             # frecuencia de muestreo (Hz)
T  = 2.0              # duración (s)
t  = np.linspace(0, T, int(Fs*T), endpoint=False)

f0 = 440.0            # "La" musical
x  = np.sin(2*np.pi*f0*t)
ruido = 0.3*np.random.randn(len(t))
y  = x + ruido

plt.figure()
plt.plot(t[:1600], y[:1600])  # primeros 0.2 s para ver detalle
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.title("Señal con ruido (fragmento)")
plt.tight_layout()
plt.show()

In [ ]:
# | fig-cap: "Espectro de magnitud (frecuencias presentes)."
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

Fs = 8000
T  = 2.0
t  = np.linspace(0, T, int(Fs*T), endpoint=False)
f0 = 440.0
y  = np.sin(2*np.pi*f0*t) + 0.3*np.random.randn(len(t))

Y   = np.fft.rfft(y)
f   = np.fft.rfftfreq(len(y), d=1/Fs)
mag = np.abs(Y)

plt.figure()
plt.plot(f, mag)
plt.xlim(0, 2000)
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud")
plt.title("‘Huella’ en frecuencia: pico cerca de 440 Hz")
plt.tight_layout()
plt.show()

:::{.notes}
Guion: Relacionar el pico con “qué tan fuerte” está ese tono en la mezcla. “Escuchar” para la compu es medir contribuciones de frecuencias.
:::

---

### 5.2 ECG sintético simplificado (5–8 latidos)

In [ ]:
# | fig-cap: "ECG sintético con latidos y leve ruido."
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

Fs = 250
dur = 8.0
t = np.linspace(0, dur, int(Fs*dur), endpoint=False)

# Ritmo ~70 lpm (periodo ~0.857 s)
rr = 60/70
signal = np.zeros_like(t)

# Modelar QRS con una gaussiana angosta y P/T suaves
def gauss(t, mu, sigma, amp):
    return amp*np.exp(-0.5*((t-mu)/sigma)**2)

for k in range(int(dur/rr)+2):
    tc = k*rr
    signal += gauss(t, tc-0.2, 0.04, 0.12)   # onda P
    signal += gauss(t, tc,      0.02, 1.2)   # QRS
    signal += gauss(t, tc+0.3,  0.06, 0.25)  # onda T

# Ruido blanco leve
signal += 0.03*np.random.randn(len(t))

plt.figure()
plt.plot(t, signal)
plt.xlabel("Tiempo (s)")
plt.ylabel("mV (relativo)")
plt.title("ECG sintético (~70 lpm)")
plt.xlim(0, dur)
plt.tight_layout()
plt.show()

:::{.notes}
Guion: Identificar “picos” (QRS) como los aplausos fuertes en un concierto: marcan el ritmo. Aclarar que en clínica real hay más detalles y calibración.
:::

---

### 5.3 EEG tipo banda alfa (\~10 Hz intermitente)

In [ ]:
# | fig-cap: "EEG sintético: ruido 1/f + ráfagas ~10 Hz (alfa)."
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

Fs = 250
dur = 5.0
N = int(Fs*dur)

# Ruido blanco -> modelado de ruido 1/f en frecuencia
white = np.random.randn(N)
W = np.fft.rfft(white)
f = np.fft.rfftfreq(N, d=1/Fs)
pink_filter = np.ones_like(f)
pink_filter[1:] = 1/np.sqrt(f[1:])  # atenuar altas frecuencias
P = W * pink_filter
pink = np.fft.irfft(P, n=N)

# Componente alfa (10 Hz) en ráfagas
t = np.arange(N)/Fs
alpha = np.sin(2*np.pi*10*t)
envelope = ((t>1) & (t<2)) | ((t>3.2) & (t<4.4))
alpha *= envelope.astype(float)

eeg = 0.7*pink/np.max(np.abs(pink)) + 0.6*alpha

plt.figure()
plt.plot(t, eeg)
plt.xlabel("Tiempo (s)")
plt.ylabel("µV (relativo)")
plt.title("EEG sintético con actividad alfa intermitente")
plt.xlim(0, dur)
plt.tight_layout()
plt.show()

:::{.notes}
Guion: Explicar que la actividad cerebral tiene “bandas” de frecuencia; la alfa (\~8–13 Hz) puede aparecer cuando estamos relajados con ojos cerrados. Esto es una caricatura útil, no un registro clínico.
:::

---

### 5.4 Procesamiento de imagen (desenfoque y realce)

In [ ]:
# | fig-cap: "Imagen de prueba (degradado + cuadros)."
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

H, W = 256, 256
# Degradado horizontal
grad = np.tile(np.linspace(0, 1, W), (H,1))
# Agregar cuadros
img = grad.copy()
img[60:120, 60:120] = 1.0
img[140:200, 140:200] = 0.2

plt.figure()
plt.imshow(img, cmap="gray", vmin=0, vmax=1)
plt.axis("off")
plt.title("Original (sintética)")
plt.tight_layout()
plt.show()

In [ ]:
# | fig-cap: "Realce (unsharp mask) a partir de desenfoque caja 3×3."
import numpy as np
import matplotlib.pyplot as plt

def box_blur(im):
    k = np.ones((3,3), dtype=float)/9.0
    H, W = im.shape
    pad = np.pad(im, 1, mode="edge")
    out = np.zeros_like(im)
    for i in range(H):
        for j in range(W):
            out[i,j] = np.sum(pad[i:i+3, j:j+3]*k)
    return out

# Partir de la misma imagen generada (repetimos por reproducibilidad)
H, W = 256, 256
grad = np.tile(np.linspace(0, 1, W), (H,1))
img = grad.copy()
img[60:120, 60:120] = 1.0
img[140:200, 140:200] = 0.2

blur = box_blur(img)
# Unsharp mask: realce = original + α*(original - blur)
alpha = 1.0
sharpen = np.clip(img + alpha*(img - blur), 0, 1)

plt.figure()
plt.imshow(sharpen, cmap="gray", vmin=0, vmax=1)
plt.axis("off")
plt.title("Procesada (realce a partir de desenfoque)")
plt.tight_layout()
plt.show()

:::{.notes}
Guion: “Desenfocar” promedia vecinos (quita detalle y ruido); el *unsharp mask* resalta bordes sumando el contraste perdido. Pedir que comparen bordes del cuadro.
:::

---

## 6) Proyectos cotidianos (idea conceptual)

* **Reconocimiento básico de patrones en voz:** distinguir entre vocales largas/cortas por su “huella” en frecuencia.
* **Clasificación de imágenes simples:** identificar un símbolo (corazón, estrella) por su contorno.
* **Apps del día a día:** reducción de ruido en llamadas, estabilización de video, filtros de cámara.

:::{.notes}
Actividad breve: mostrar dos espectros (uno con pico \~440 Hz, otro \~880 Hz) y preguntar cuál es más “agudo”. Objetivo: enlazar percepción con representación.
:::

---

## 7) Proyectos biomédicos (propósito y ética)

* **ECG:** apoyar la detección de ritmos irregulares para alertar a tiempo.
* **EEG:** estudiar estados de sueño o episodios que requieren atención.
* **Imágenes médicas:** resaltar estructuras (hueso, tejido blando) para guiar decisiones.

**Cautelas éticas**

* Los algoritmos **no reemplazan** a profesionales.
* Debe haber **validación clínica** y supervisión experta.
* Evitar **sesgos** que afecten a grupos poblacionales.

:::{.notes}
Guion: Subrayar que la tecnología es una lupa: amplifica, no decide por sí sola. Citar privacidad y consentimiento.
:::

---

## 8) Limitaciones y ética

* **Datos sesgados** → resultados injustos.
* **Privacidad**: proteger información personal y de pacientes.
* **Consentimiento informado**: especialmente con imágenes de personas.
* **Transparencia**: explicar qué hace el sistema y sus límites.

:::{.notes}
Idea clave: “Si no está bien medido, no estará bien decidido”. Pregunta: ¿qué riesgos ven al entrenar con pocas personas de ciertas edades o regiones?
:::

---

## 9) Recursos e **imágenes libres** (repositorios y créditos)

**Repositorios verificados**

* **Wikimedia Commons** (metadatos/licencias claros).
* **Unsplash** (Licencia Unsplash: uso libre, atribución no obligatoria; recomendable acreditar autor).
* **Pixabay** (CC0/PD, según archivo).

**Imágenes sugeridas para carpeta `images/` (descargar y acreditar):**

* `images/Normal_ECG_2.svg` — *Normal ECG 2* (CC BY-SA 3.0). Wikimedia Commons. ([Wikimedia Commons][1])
* `images/MRI_Brain_T1_Axial_11.jpg` — *MRI Brain T1 Axial (11)* (CC BY-SA 4.0). Wikimedia Commons. ([Wikimedia Commons][2])
* `images/Ultrasound_image_110316103934_1045590.jpg` — Ecografía (CC BY-SA 3.0) por **Nevit Dilmen**. Wikimedia Commons. ([Wikimedia Commons][3])
* (Opcional) Foto contextual sin pacientes: *Doctor using stethoscope…* (Licencia Unsplash). Guardar como `images/unsplash_stethoscope.jpg`. ([Unsplash][4])

**Nota sobre consentimiento y pacientes en imágenes:** ver guía de Commons sobre imágenes de pacientes. ([Wikimedia Commons][5])

:::{.notes}
Instrucciones: Verificar en cada página de Commons la licencia exacta y texto de atribución. En Unsplash, aunque no es obligatorio, incluir nombre del autor y enlace.
:::

---

## 10) Cierre y preguntas

**Ideas para proyectos escolares breves**

* “Mi barrio suena”: medir ruido con el celular y graficar cómo cambia a lo largo del día.
* “Foto más clara”: comparar imagen original vs. procesada con el realce mostrado.
* “Ritmo saludable”: simular un ECG y medir el intervalo entre picos.

**Mensaje final**
Procesar señales e imágenes es *aprender a leer historias* que cuentan los datos para tomar mejores decisiones.

:::{.notes}
Guion: Abrir turno de preguntas. Proponer que formen equipos y elijan un mini-proyecto con datos accesibles y sin información sensible.
:::

---

## 11) Referencias (introductorias y de divulgación)

* Mneney, S. H. **An Introduction to Digital Signal Processing: A Focus on Implementation** (Open Access). OAPEN. (2008). ([library.oapen.org][6])
* IEEE Signal Processing Society. **Signal Processing 101** (página divulgativa). ([IEEE Signal Processing Society][7])
* Li, M. et al. **Medical image analysis using deep learning algorithms**. *Electronics*, 2023 (OA, PMC). ([PMC][8])
* Hadaeghi, F. **An Introduction to Biomedical Signal Processing (Lecture Notes)** (OA PDF). ([ai.rug.nl][9])
* (En verificación) Capítulos introductorios de *Biomedical Image Processing* (Deserno). Revisión en PMC. Alternativas reales:

  * *Medical Image Analysis* (Elsevier, opciones OA). ([ScienceDirect][10])
  * *Biomedical Signal Processing and Control* (Elsevier, opciones OA). ([ScienceDirect][11])

:::{.notes}
Aclaración: fuentes seleccionadas por acceso abierto y carácter introductorio. Si alguna queda inaccesible, mantener como “en verificación” y usar alternativas OA listadas.
:::

---

## Notas de producción (para el docente)

* **Base utilizada (`TallerPrimerSemestre.qmd`)**: se revisaron estructura narrativa, tono de bienvenida y algunas ideas para dinámicas. No se reutilizaron celdas con dependencias o estilos no requeridos, ni se mantuvieron secciones que excedían el nivel 10–11 o requerían matemáticas. Se ajustó cohesión y se añadió la secuencia exigida (1–11) y los cuatro bloques Python con **matplotlib** exclusivamente.
* **Reproducibilidad**: cada bloque fija `np.random.seed(42)` y evita librerías externas.
* **Imágenes**: descargar a `images/` con los nombres indicados y conservar créditos/licencias en la diapositiva 9.

:::{.notes}
Checklist del docente antes de presentar:

1. Ejecutar todas las celdas (`quarto render`) y verificar que cada figura aparece en <2 s.
2. Confirmar que los archivos en `images/` existen con los nombres propuestos.
3. Mantener visible la diapositiva 9 para aclarar licencias si se pregunta.
4. Preparar 1–2 preguntas detonantes extra según el grupo.
   :::

---

## Referencias

[1]: https://commons.wikimedia.org/wiki/File%3ANormal_ECG_2.svg?utm_source=chatgpt.com "File:Normal ECG 2.svg - Wikimedia Commons"
[2]: https://commons.wikimedia.org/wiki/File%3AMRI_Brain_T1_Axial_%2811%29.jpg?utm_source=chatgpt.com "File:MRI Brain T1 Axial (11).jpg"
[3]: https://commons.wikimedia.org/wiki/File%3AUltrasound_image_110316103934_1045590.jpg?utm_source=chatgpt.com "File:Ultrasound image 110316103934 1045590.jpg"
[4]: https://unsplash.com/photos/doctor-using-stethoscope-examination-his-patient-in-hospital-BXU2Z_YXJoo?utm_source=chatgpt.com "Doctor using stethoscope examination his patient in hospital."
[5]: https://commons.wikimedia.org/wiki/Commons%3APatient_images?utm_source=chatgpt.com "Commons:Patient images"
[6]: https://library.oapen.org/bitstream/handle/20.500.12657/59716/9781000793888.pdf?isAllowed=y&sequence=1&utm_source=chatgpt.com "An Introduction to Digital Signal Processing: A Focus on ..."
[7]: https://signalprocessingsociety.org/our-story/signal-processing-101?utm_source=chatgpt.com "Signal Processing 101"
[8]: https://pmc.ncbi.nlm.nih.gov/articles/PMC10662291/?utm_source=chatgpt.com "Medical image analysis using deep learning algorithms - PMC"
[9]: https://www.ai.rug.nl/minds/uploads/Chapter%201_introduction.pdf?utm_source=chatgpt.com "An Introduction to Biomedical Signal Processing Lecture ..."
[10]: https://www.sciencedirect.com/journal/medical-image-analysis/publish/open-access-options?utm_source=chatgpt.com "Open access information - Medical Image Analysis"
[11]: https://www.sciencedirect.com/journal/biomedical-signal-processing-and-control/publish/open-access-options?utm_source=chatgpt.com "Biomedical Signal Processing and Control - ISSN 1746-8094"